# Building LLM applications: Notebook 06

# LangGraph Agent

## Initialize

In [1]:
import os
import dotenv

from IPython.display import Image, display

from langchain_core.tools import tool
from langchain_core.messages import HumanMessage, SystemMessage

from langgraph.prebuilt import ToolNode

from langgraph.graph import MessagesState
from langgraph.graph import StateGraph, START, END

from langchain_ollama import ChatOllama

In [2]:
MODEL = 'llama3.2:3b-instruct-fp16'

In [3]:
# Read fro `.env` file
dotenv.load_dotenv()

OLLAMA_URL = os.getenv('OLLAMA_URL')
print(f"Using Ollama server: {OLLAMA_URL if OLLAMA_URL else 'local'}")

Using Ollama server: local


## Graph: Agent = LLM + tools

In [4]:
@tool
def add(a: float, b: float) -> float:
    """Add two numbers"""
    print(f"Adding {a} + {b}")
    return a + b


@tool
def sub(a: float, b: float) -> float:
    """Substract two numbers"""
    print(f"Substracting {a} - {b}")
    return a - b


@tool
def multiply(a: float, b: float) -> float:
    """Multiply two numbers"""
    print(f"Multiply {a} * {b}")
    return a * b


@tool
def divide(a: float, b: float) -> float:
    """Divide two numbers"""
    print(f"Divide {a} * {b}")
    return a / b


tools = [add, sub, multiply, divide]

In [5]:
# Create the LLM with tools
llm = ChatOllama(model=MODEL, base_url=OLLAMA_URL)
llm_with_tools = llm.bind_tools(tools)

In [6]:
state = MessagesState()

In [7]:
# System message
sys_msg = SystemMessage(content="Help me do some math. Use the tools avilably for the operations.")

# Node assistnant
def node_assistant(state: MessagesState):
   """ If the latest LLM response has a tool, we will use it """
   messages = state["messages"]
   ret = llm_with_tools.invoke(messages)
   return {"messages": ret}

# A node that can handle tools
node_tools = ToolNode(tools)

In [8]:
# Build graph
builder = StateGraph(MessagesState)
builder.add_node("assistant", node_assistant)
builder.add_node("tools", node_tools)

In [9]:
def should_run_tools(state: MessagesState):
    """ If the latest message has tool calls, then run the tools """
    messages = state['messages']
    latest_message = messages[-1]
    if hasattr(latest_message, "tool_calls") and len(latest_message.tool_calls) > 0:
        return "tools"
    return END

In [10]:
# Edges
builder.add_edge(START, "assistant")
builder.add_conditional_edges("assistant", should_run_tools)
builder.add_edge("tools", "assistant")

In [ ]:
graph = builder.compile()

# View
display(Image(graph.get_graph().draw_mermaid_png()))

### Run it

In [ ]:
messages = [HumanMessage(content="Add 3 and 39")]

ret = graph.invoke({"messages": messages})

# The state is returned, we show the messages
for m in ret["messages"]:
    m.pretty_print()